In [1]:
import tensorflow as tf
import numpy as np
from textloader import TextLoader
import matplotlib.pyplot as plt
from tensorflow.python.ops.rnn_cell import BasicLSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.python.ops.rnn_cell import RNNCell

In [ ]:
class mygru( RNNCell ):
 
    def __init__( self, num_units, input_size=None):
        self.num_units = num_units
        self.input_size = input_size
 
    @property
    def state_size(self):
        return self.num_units
 
    @property
    def output_size(self):
        return self.num_units
 
    def __call__( self, inputs, state, scope=None ):
        # h should be [batch_size, num_units]
        # state should be [batch_size, num_units]
        with tf.variable_scope("GRU_cell") as scope:
            init = tf.contrib.layers.variance_scaling_initializer()
            Wz = tf.get_variable("W_z", [inputs.get_shape()[1], self.num_units], initializer=init)
            Uz = tf.get_variable("U_z", [state.get_shape()[1], self.num_units], initializer=init)
            bz = tf.get_variable("b_z", [self.num_units], initializer=init)
            zt = tf.sigmoid(tf.nn.bias_add(tf.matmul(inputs, Wz) + tf.matmul(state, Uz), bz)) # tf.nn.bias_add() ??
            Wr = tf.get_variable("W_r", [inputs.get_shape()[1], self.num_units], initializer=init)
            Ur = tf.get_variable("U_r", [state.get_shape()[1], self.num_units], initializer=init)
            br = tf.get_variable("b_r", [self.num_units], initializer=init)
            rt = tf.sigmoid(tf.nn.bias_add(tf.matmul(inputs, Wr) + tf.matmul(state, Ur), br))
            Wh = tf.get_variable("W_h", [inputs.get_shape()[1], self.num_units], initializer=init)
            Uh = tf.get_variable("U_h", [state.get_shape()[1], self.num_units], initializer=init)
            bh = tf.get_variable("b_h", [self.num_units], initializer=init)
            intermediate = tf.nn.bias_add(tf.matmul(inputs, Wh) + tf.matmul(tf.multiply(rt, state), Uh), bh)
            ht = tf.multiply(zt, state) + tf.multiply(1.-zt, tf.tanh(intermediate))
            return ht, ht

In [ ]:
# Global variables
batch_size = 50
sequence_length = 50
state_dim = 128
num_layers = 2

data_loader = TextLoader( ".", batch_size, sequence_length )
vocab_size = data_loader.vocab_size  # dimension of one-hot encodings

tf.reset_default_graph()

# ==================================================================
# ==================================================================

# define placeholders for our inputs.  
# in_ph is assumed to be [batch_size, sequence_length]
# targ_ph is assumed to be [batch_size, sequence_length]

in_ph = tf.placeholder( tf.int32, [ batch_size, sequence_length ], name='inputs' )
targ_ph = tf.placeholder( tf.int32, [ batch_size, sequence_length ], name='targets' )
in_onehot = tf.one_hot( in_ph, vocab_size, name="input_onehot" )

inputs = tf.split( in_onehot, sequence_length, axis=1 )
inputs = [tf.squeeze(i, [1]) for i in inputs] # inputs is list length sequence_length; each element is [batch_size, vocab_size]
targets = tf.split( targ_ph, sequence_length, axis=1, ) # targets is list length sequence_length; each element of targets is 1D vector length batch_size

# ------------------
# COMPUTATION GRAPH

with tf.variable_scope("Computation_Graph") as scope:
    basic1 = mygru(state_dim) # If I weren't using my own GRU class, I would call BasicLSTMCell here instead
    basic2 = mygru(state_dim) # If I weren't using my own GRU class, I would call BasicLSTMCell here instead
    rnn = MultiRNNCell([basic1,basic2])
    initial_state = rnn.zero_state(batch_size, tf.float32) # initial_state is a list of tensors
    output_list, final_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, rnn)
    
    init = tf.contrib.layers.variance_scaling_initializer()
    W = tf.Variable(tf.random_normal([state_dim, vocab_size], stddev=0.02))
    b = tf.Variable(tf.random_normal([vocab_size], stddev=0.01))
    logits = [tf.matmul(output, W) + b for output in output_list]

    weights = [1.]*sequence_length
    loss = tf.contrib.legacy_seq2seq.sequence_loss(logits, targets, weights)

    c = 0.001
    optim = tf.train.AdamOptimizer(learning_rate=c).minimize(loss)
    
# ------------------
# SAMPLER GRAPH

# reuse the parameters of the cell and the parameters you used to transform state outputs to logits!

batch_size = 1
sequence_length = 1
s_in_ph = tf.placeholder( tf.int32, [batch_size], name='inputs' )
s_in_onehot = tf.one_hot( s_in_ph, vocab_size, name="input_onehot" )

s_inputs = tf.split( s_in_onehot, sequence_length, axis=0 )

with tf.variable_scope("Sampler") as scope:
    s_initial_state = rnn.zero_state(batch_size, tf.float32) # initial_state is a list of tensors

    s_output_list, s_final_state = tf.contrib.legacy_seq2seq.rnn_decoder(s_inputs, s_initial_state, rnn)
    
    s_probs = [tf.matmul(output, W) + b for output in s_output_list] # W, b from previous RNN

# ==================================================================
# ==================================================================

def sample( num=200, prime='ab' ):
    # prime the pump 

    # generate an initial state. this will be a list of states, one for
    # each layer in the multicell.
    s_state = sess.run( s_initial_state )

    # for each character, feed it into the sampler graph and
    # update the state.
    for char in prime[:-1]:
        x = np.ravel( data_loader.vocab[char] ).astype('int32')
        feed = { s_in_ph:x }
        for i, s in enumerate( s_initial_state ):
            feed[s] = s_state[i]
        s_state = sess.run( s_final_state, feed_dict=feed )

    # now we have a primed state vector; we need to start sampling.
    ret = prime
    char = prime[-1]
    for n in xrange(num):
        x = np.ravel( data_loader.vocab[char] ).astype('int32')

        # plug the most recent character in...
        feed = { s_in_ph:x }
        for i, s in enumerate( s_initial_state ):
            feed[s] = s_state[i]
        ops = [s_probs]
        ops.extend( list(s_final_state) )

        retval = sess.run( ops, feed_dict=feed )

        s_probsv = retval[0]
        s_state = retval[1:]

        # ...and get a vector of probabilities out!

        # now sample (or pick the argmax)
        sample = np.argmax( s_probsv[0] )
        # sample = np.random.choice( vocab_size, p=s_probsv[0] )

        pred = data_loader.chars[sample]
        ret += pred
        char = pred

    return ret

# ==================================================================
# ==================================================================

sess = tf.Session()
sess.run( tf.global_variables_initializer() )
summary_writer = tf.summary.FileWriter( "./tf_logs", graph=sess.graph )
print "FOUND %d BATCHES" % data_loader.num_batches

for j in xrange(200):

    state = sess.run( initial_state )
    data_loader.reset_batch_pointer()

    for i in xrange( data_loader.num_batches ):
        
        x,y = data_loader.next_batch()

        # we have to feed in the individual states of the MultiRNN cell
        feed = { in_ph: x, targ_ph: y }
        for k, s in enumerate( initial_state ):
            feed[s] = state[k]

        ops = [optim,loss]
        ops.extend( list(final_state) )

        # retval will have at least 3 entries:
        # 0 is None (triggered by the optim op)
        # 1 is the loss
        # 2+ are the new final states of the MultiRNN cell
        retval = sess.run( ops, feed_dict=feed )

        lt = retval[1]
        state = retval[2:]

        if i%1000==0:
            print "%d %d\t%.4f" % ( j, i, lt )

    print sample( num=60, prime="He " )
#    print sample( num=60, prime="ababab" )
#    print sample( num=60, prime="foo ba" )
#    print sample( num=60, prime="abcdab" )

with open("my_hp.txt", 'w') as f:
    for i in xrange(20):
        f.write(sample(num=60, prime="He ") + '\n')

summary_writer.close()

# Harry Potter and the Deathly Hallows, Chapter 36 (my text choice)

In [3]:
with open("my_hp.txt", 'r') as f:
    hp_lines = f.readlines()
for h in hp_lines:
    print(h[:-1])

He was shathere was a forced of the wand him and and the crowd 
He was beend atound his eyes at the said to the comporse he had
He was beend atound his eyes at the sale was and and the contan
He was because he loved her and couts to to kill them war arryo
He was she chanted into the air. “I’s dows donged loughter, and
He was a forted of her and cors any had no was streamed of the 
He was seath of the samper that the same to the wand was a fore
He was shathere with the had not lise that the wand in the had 
He was shathere wished a filll the castle later him him him wit
He was shathere wished a for like a might and harde to his feet
He was a forseath of him a finet time, the flain a chanted onte
He was a forced of the Elder Wand like a combort of them and ha
He was slake’s had been it and the same siled the same siled an
He was sidenst from the castle and streamed Harry as the same s
He was shathered the entrang the granted and whise to dist they
He was nower at the wand in the compone,

# Alma

In [5]:
with open("my_alma.txt", 'r') as f:
    hp_lines = f.readlines()
for h in hp_lines:
    print(h[:-1])

And because of the words of God we will repent and see that ye s
And because of the words of Amulek, and the word of God with the
And because of their words, and their country to the land of Mid
And Amulek had gone forth against themselves to the land of Zara
And Amulek, being lifted up in the land of Middoni and the serva
And of every kind.

 But behold, it was not all things, and the 
And of every kind; and they were all an athye over the land of S
And of the servants of the Lamanites who were in the land of Mid
And of the seed.

 And it came to pass that the Lamanites are a 
And of the servants of the Lamanites who were slain; for behold,
And of the servants of the Lamanites who were in the land of Mid
And of the Lamanites who had been led by the seashore, and the p
And of the traditions of their fathers, and the men of the Laman
And of the Lord were goodness.

 And now, my son, I desire that 
And of the Lord were good shercy hath great presence of the Lama
And of the seed.

 And th